In [31]:
%matplotlib inline
import os 
from collections import deque
import random
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import editdistance
import sys
import RNA
from typing import Dict, List, Tuple

import torch
from torch import nn
from tqdm import tqdm_notebook as tqdm
import torch.nn.functional as F
import torch.multiprocessing as mp
import torch.optim as optim
from torch.nn.utils import clip_grad_norm_

# import path 
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

from utils.sequence_utils import translate_one_hot_to_string,generate_random_mutant
from models.Theoretical_models import *
from models.Noise_wrapper import *
from exploration_strategies.CE import *
from utils.landscape_utils import *
from models.RNA_landscapes import *
from models.Multi_dimensional_model import *

from segment_tree import MinSegmentTree, SumSegmentTree

In [18]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
RAA="UGCA" #alphabet
length=40
wt=generate_random_sequences(length,1,alphabet=RAA)[0]
print(wt)
#make a simple folding landscape starting at wt
landscape1=RNA_landscape(wt)
noise_alpha=1
virtual_per_measure_ratio=15
temperature=0.1
# there are multiple abstract "noise models" you can use, or you can try to train your own model, using skM
noisy_landscape_CE=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
noisy_landscape_RL=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
noisy_landscape_RL_multiple=Noise_wrapper(landscape1,noise_alpha=noise_alpha)
#noisy_landscape=Gaussian_noise_landscape(base_landscape,noise_alpha=0.15)
#noisy_landscape=DF_noise_landscape(base_landscape,noise_alpha=0.5)
batch_size = 1000
initial_genotypes=list(set([wt]+[generate_random_mutant(wt,0.05,RAA) for i in range(batch_size)]))[:batch_size]

GGAAAUAAUCACACAACGAUCAUGUGGAAUAAUACAACGG


# SINGLE AGENT RL

In [77]:
import numpy as np
import random
import bisect
from utils.sequence_utils import translate_string_to_one_hot, translate_one_hot_to_string

def renormalize_moves(one_hot_input, rewards_output):
    """ensures that staying in place gives no reward"""
    zero_current_state = (one_hot_input - 1) * (-1)
    return np.multiply(rewards_output, zero_current_state)

def walk_away_renormalize_moves(one_hot_input, one_hot_wt, rewards_output):
    """ensures that moving toward wt is also not useful"""
    zero_current_state=(one_hot_input-1)*-1
    zero_wt=((one_hot_wt-1)*-1)
    zero_conservative_moves=np.multiply(zero_wt,zero_current_state)
    return np.multiply(rewards_output,zero_conservative_moves)

def get_all_singles_fitness(model,sequence,alphabet):
    prob_singles=np.zeros((len(alphabet),len(sequence)))
    for i in range(len(sequence)):
        for j in range(len(alphabet)):
            putative_seq=sequence[:i]+alphabet[j]+sequence[i+1:]
           # print (putative_seq)
            prob_singles[j][i]=model.get_fitness(putative_seq)
    return prob_singles

def get_all_mutants(sequence):
    mutants = []
    for i in range(sequence.shape[0]):
        for j in range(sequence.shape[1]):
            putative_seq = sequence.copy()
            putative_seq[:, j] = 0
            putative_seq[i, j] = 1
            mutants.append(putative_seq)
    return np.array(mutants)

def sample_greedy(matrix):
    i,j=matrix.shape
    max_arg=np.argmax(matrix)
    y=max_arg%j
    x=int(max_arg/j)
    output=np.zeros((i,j))
    output[x][y]=matrix[x][y]
    return output

def sample_multi_greedy(matrix):
    n = 5 # the number of base positions to greedily change
    max_args = np.argpartition(matrix.flatten(), -n)[-n:]
    i,j=matrix.shape
    output=np.zeros((i,j))
    for max_arg in max_args:
        y=max_arg%j
        x=int(max_arg/j)
        output[x][y]=matrix[x][y]
    return output

def sample_random(matrix):
    i,j=matrix.shape
    non_zero_moves=np.nonzero(matrix)
   # print (non_zero_moves)
    k=len(non_zero_moves)
    l=len(non_zero_moves[0])
    if k!=0 and l!=0:
        rand_arg=random.choice([[non_zero_moves[alph][pos] for alph in range(k)] for pos in range(l)])
    else:
        rand_arg=[random.randint(0,i-1),random.randint(0,j-1)]
    #print (rand_arg)
    y=rand_arg[1]
    x=rand_arg[0]
    output=np.zeros((i,j))
    output[x][y] = 1
    return output   

def action_to_scalar(matrix):
    matrix = matrix.ravel()
    for i in range(len(matrix)):
        if matrix[i] != 0:
            return i
    
def construct_mutant_from_sample(pwm_sample, one_hot_base):
    one_hot = np.zeros(one_hot_base.shape)
    one_hot += one_hot_base
    nonzero = np.nonzero(pwm_sample)
    nonzero = list(zip(nonzero[0], nonzero[1]))
    for nz in nonzero: # this can be problematic for non-positive fitnesses
        i, j = nz
        one_hot[:,j]=0
        one_hot[i,j]=1
    return one_hot

def best_predicted_new_gen(actor, genotypes, alphabet, pop_size):
    mutants = get_all_mutants(genotypes)
    one_hot_mutants = np.array([translate_string_to_one_hot(mutant, alphabet) for mutant in mutants])
    torch_one_hot_mutants = torch.from_numpy(np.expand_dims(one_hot_mutants, axis=0)).float()
    predictions = actor(torch_one_hot_mutants)
    predictions = predictions.detach().numpy()
    best_pred_ind = predictions.argsort()[-pop_size:]
    return mutants[best_pred_ind]

def make_one_hot_train_test(genotypes, model, alphabet):
    genotypes_one_hot = np.array([translate_string_to_one_hot(genotype, alphabet) for genotype in genotypes])
    genotype_fitnesses = []
    for genotype in genotypes:
        genotype_fitnesses.append(model.get_fitness(genotype))
    genotype_fitnesses = np.array(genotype_fitnesses)

    return genotypes_one_hot, genotype_fitnesses

## Test Effectiveness of Model

In [78]:
import keras 
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv1D, BatchNormalization, Flatten
from keras.regularizers import l1, l2
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso

# such fraud, this is literally Ridge Regression
model = Sequential()
model.add(Dense(40, activation='relu', input_shape=(160,)))
model.add(Dense(1, activation='relu', input_shape=(160,)))
#model.add(Dense(1, activation='sigmoid', input_shape=(160,)))

print(model.summary())

def build_genotype_data(samples):
    genotypes = list(set([wt] + [generate_random_mutant(wt, 0.05, RAA) for i in range(samples)]))
    genotypes_one_hot, genotype_fitnesses = make_one_hot_train_test(genotypes, noisy_landscape_RL, RAA)
    genotypes_one_hot = genotypes_one_hot.reshape((-1, 160))
    #genotypes_one_hot = np.transpose(genotypes_one_hot, (0, 2, 1))
    return genotypes_one_hot, genotype_fitnesses

train_data, train_labels = build_genotype_data(60000)
test_data, test_labels = build_genotype_data(5000)
model.compile(loss=keras.losses.mean_squared_error,
              optimizer=keras.optimizers.Adam(),
              metrics=['mse'])
model.fit(train_data, train_labels,
          batch_size=128,
          epochs=10,
          verbose=1,
          validation_data=(test_data, test_labels))
preds = model.predict(test_data)
print('NN MSE', mean_squared_error(preds, test_labels))

# lasso regression benchmark 
lasso = Lasso()
lasso.fit(train_data, train_labels)
preds = lasso.predict(test_data)
print('Lasso MSE', mean_squared_error(preds, test_labels))

# random forest benchmark
rf = RandomForestRegressor()
rf.fit(train_data, train_labels)
preds = rf.predict(test_data)
print('Random Forest MSE', mean_squared_error(preds, test_labels))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 40)                6440      
_________________________________________________________________
dense_10 (Dense)             (None, 1)                 41        
Total params: 6,481
Trainable params: 6,481
Non-trainable params: 0
_________________________________________________________________
None
Train on 17448 samples, validate on 2219 samples
Epoch 1/10
17448/17448 [==============================] - 1s 67us/step - loss: 0.0056 - mean_squared_error: 0.0056 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 2/10
17448/17448 [==============================] - 1s 33us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0017 - val_mean_squared_error: 0.0017
Epoch 3/10
17448/17448 [==============================] - 0s 26us/step - loss: 0.0017 - mean_squared_error: 0.0017 - val_loss: 0.0017 - val_mean_squ

/home/alexander/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Random Forest MSE 4.947542827406301e-05


In [66]:
from collections import deque
# from utils.RL_utils import *
from utils.sequence_utils import translate_one_hot_to_string,generate_random_mutant
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

class Q_Network(nn.Module):
    def __init__(self, sequence_len, alphabet_len):
        super(Q_Network, self).__init__()
        self.sequence_len = sequence_len
        self.alphabet_len = alphabet_len
        self.linear1 = nn.Linear(alphabet_len * sequence_len, alphabet_len * sequence_len)
        self.bn1 = nn.BatchNorm1d(alphabet_len * sequence_len)
        self.linear2 = nn.Linear(alphabet_len * sequence_len, sequence_len)
        self.bn2 = nn.BatchNorm1d(sequence_len)
        self.linear3 = nn.Linear(sequence_len, sequence_len)
        self.bn3 = nn.BatchNorm1d(sequence_len)
        self.linear4 = nn.Linear(sequence_len, alphabet_len * sequence_len)
    
    def forward(self, x):
        x = x.view(-1, self.alphabet_len * self.sequence_len)
        x = self.bn1(F.relu(self.linear1(x)))
        x = self.bn2(F.relu(self.linear2(x)))
        x = self.bn3(F.relu(self.linear3(x)))
        x = F.relu(self.linear4(x))
        return x
    
def build_q_network(sequence_len, alphabet_len, device):
    model = Q_Network(sequence_len, alphabet_len).to(device)
    print(model)
    return model


class RL_agent_DQN():
    '''
    Based off https://colab.research.google.com/drive/1NsbSPn6jOcaJB_mp9TmkgQX7UrRIrTi0
    '''
    def __init__(self, start_sequence, alphabet, gamma=0.9, 
                 memory_size=1000, batch_size=1000, experiment_batch_size=1000,
                 device = "cpu", noise_alpha=1):
        self.alphabet = alphabet
        self.state = translate_string_to_one_hot(start_sequence, self.alphabet)
        self.seq_size = len(start_sequence)
        self.q_network = build_q_network(self.seq_size, len(self.alphabet), device)
        self.start_sequence = translate_string_to_one_hot(start_sequence,self.alphabet)
        self.memory_size = memory_size
        self.gamma = gamma
        self.batch_size = batch_size
        self.experiment_batch_size = experiment_batch_size
        self.memory = []
        self.seen_sequences = []
        self.landscape = Noise_wrapper(RNA_landscape(start_sequence), 
                                       noise_alpha=noise_alpha, always_costly=True)
        self.best_fitness = 0

    def reset_position(self,sequence):
        self.state=translate_string_to_one_hot(sequence,self.alphabet)

    def get_position(self):
        return translate_one_hot_to_string(self.state,self.alphabet)

    def translate_pwm_to_sequence(self,input_seq_one_hot,output_pwm):
        diff=output_pwm-input_seq_one_hot
        most_likely=np.argmax(diff,axis=0)
        out_seq=""
        for m in most_likely:
            out_seq+=self.alphabet[m]
        return out_seq
    
    def sample(self):
        indices = np.random.choice(len(self.memory), self.batch_size)
        rewards, actions, states, next_states = zip(*[self.memory[ind] for ind in indices])
        return np.array(rewards), np.array(actions), np.array(states), np.array(next_states) 
    
    def q_network_loss(self, batch, device="cpu"):
        """
        Calculate MSE between actual state action values,
        and expected state action values from DQN
        """
        rewards, actions, states, next_states = batch

        rewards_v = torch.tensor(rewards).float()
        actions_v = torch.tensor(actions).float()
        states_v = torch.tensor(states).float()
        next_states_v = torch.tensor(next_states).float()

        state_action_values = self.q_network(states_v).gather(1, actions_v.long().unsqueeze(-1)).squeeze(-1)
        next_state_values = self.q_network(next_states_v).max(1)[0]
        next_state_values = next_state_values.detach()
        expected_state_action_values = next_state_values * self.gamma + rewards_v
        
        return nn.MSELoss()(state_action_values, expected_state_action_values)

    def train_actor(self, train_epochs=10):
        total_loss = 0.
        # train Q network on new samples 
        optimizer = optim.Adam(self.q_network.parameters())
        for epoch in range(train_epochs):
            batch = self.sample()
            optimizer.zero_grad()
            loss = self.q_network_loss(batch)
            loss.backward()
            clip_grad_norm_(self.q_network.parameters(), 1.0, norm_type=1)
            optimizer.step()
            total_loss += loss.item()
        return (total_loss / train_epochs)

    def pick_action(self, epsilon):
        prediction = self.q_network(torch.tensor(self.state).float()).detach().numpy()[0]
        prediction = prediction.reshape((len(self.alphabet), self.seq_size))
        # make action
        moves = renormalize_moves(self.state, prediction)
        p = random.random()
        action = sample_random(moves) if p < epsilon else sample_greedy(moves)
        # get next state (mutant)
        mutant = construct_mutant_from_sample(action, self.state)
        mutant_string = translate_one_hot_to_string(mutant, self.alphabet)
        self.state = mutant

        return action_to_scalar(action), mutant
    
    def run_RL(self, generations=10, train_epochs=10, epsilon_min=0.1):
        self.q_network.eval()
        while self.landscape.cost < self.experiment_batch_size*generations:
            eps = max(epsilon_min, (0.5 - self.landscape.cost / (self.experiment_batch_size * generations)))
            b = 0
            new = []
            while(b < self.experiment_batch_size):
                state = self.state.copy() 
                action, new_state = agent.pick_action(eps)
                new_state_string = translate_one_hot_to_string(new_state, self.alphabet)
                reward = self.landscape.get_fitness(new_state_string)
                if not new_state_string in self.landscape.measured_sequences:
                    if reward > self.best_fitness:
                        prediction = self.q_network(torch.tensor(self.state).float()).detach().numpy()[0]
                        print(prediction)
                    self.best_fitness = max(self.best_fitness, reward)
                    self.memory.append((reward, action, state, new_state))
                    b += 1
            
            self.memory = sorted(self.memory, key = lambda x: x[0])
            avg_loss = agent.train_actor(train_epochs)
            print (self.landscape.cost, self.memory[-1][0], avg_loss)
            
        return (translate_one_hot_to_string(self.memory[-1][2], self.alphabet), self.memory[-1][0])

In [61]:
batch_size = 1000
generations = 20
agent = RL_agent_DQN(wt, alphabet=RAA, gamma=0.8, memory_size=10000, device=device)
agent.run_RL(generations=generations, train_epochs=40)

Q_Network(
  (linear1): Linear(in_features=160, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=40, bias=True)
  (bn3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=40, out_features=160, bias=True)
)
[0.01199206 0.         0.         0.         0.         0.
 0.08858879 0.         0.         0.         0.         0.
 0.07998653 0.         0.09238742 0.         0.         0.
 0.         0.16831385 0.12805128 0.         0.         0.
 0.         0.11515898 0.07552151 0.14304395 0.         0.
 0.06086306 0.         0.         0.         0.13619223 0.01921608
 0.06052118 0.10852489 0.         0.         0.         0.20850419
 0.         0.         0.     

[0.00747735 0.         0.         0.         0.         0.
 0.08549735 0.         0.         0.         0.         0.
 0.08255263 0.         0.09161044 0.         0.         0.
 0.         0.16527669 0.12007603 0.         0.         0.
 0.         0.11481929 0.08361248 0.13519627 0.         0.
 0.05107372 0.         0.         0.         0.14284876 0.00853532
 0.05946513 0.10554743 0.         0.         0.         0.20608889
 0.         0.         0.         0.13036308 0.1227404  0.
 0.         0.04512655 0.06976716 0.         0.06469391 0.
 0.0529283  0.         0.03829642 0.07017108 0.00693429 0.138284
 0.         0.         0.         0.         0.         0.
 0.05343891 0.         0.16975415 0.08892184 0.         0.
 0.         0.0927977  0.         0.         0.10061795 0.
 0.         0.         0.         0.03157353 0.17851898 0.
 0.02037353 0.01383606 0.12796767 0.         0.04887567 0.08403586
 0.1360181  0.0869879  0.01174138 0.         0.13025716 0.
 0.         0.         0. 

[0.6423153  0.0731511  0.         0.2681135  0.         0.7756563
 0.96529067 0.         0.21500847 0.         0.         0.
 0.8822986  0.03058804 0.61795884 0.         0.02124105 0.
 0.         1.0937057  1.0311192  0.         0.28756136 0.
 0.         0.72705287 1.0777934  0.8706764  0.         0.
 0.96934533 0.         0.         0.24989522 1.039292   0.41088244
 1.0364716  0.99890685 0.35959703 0.         0.         0.9816183
 0.         0.         0.         0.87041104 1.0817384  0.
 0.42604834 0.9429854  0.813498   0.         0.8129555  0.38046992
 1.1031381  0.8561396  0.3083395  0.02387876 0.00489601 1.0866134
 0.         0.         0.06564234 0.11614087 0.         0.
 0.8338748  0.         0.663537   1.1041411  0.24225199 0.
 0.10692825 1.119786   0.         0.         0.6733028  0.
 0.3421929  0.         0.46272552 0.89017385 0.8114488  0.35592055
 0.7059634  0.5559345  1.0779103  0.         0.842047   0.5479688
 1.0838499  0.58821523 0.6161508  0.0747254  0.50327754 0.
 0.2

3270 0.2799999910242417 0.04829944209195673
[7.5658917 7.4151073 7.7216864 7.356265  0.        7.7811747 7.3661885
 7.669026  7.5881085 0.        0.        0.        7.751442  7.602016
 6.914665  0.        7.735486  0.        0.        7.4149547 7.5137296
 0.        7.325353  0.        7.456982  7.3913827 7.3912053 7.6388307
 0.        0.        7.4792223 7.6378417 0.        7.745514  7.286145
 7.8942475 7.6330824 7.819835  7.7842937 4.0694256 0.        7.82092
 0.        0.        0.        7.368512  7.6963997 0.        7.755881
 7.99707   7.818747  0.        7.598029  7.835789  8.199409  7.996763
 7.478006  7.582471  7.5665708 7.849188  0.        0.        7.6129227
 7.89032   7.8556223 0.        7.707964  0.        7.7666554 7.847556
 8.09915   0.        7.952007  7.619453  0.        0.        7.5481224
 0.        7.512569  0.        7.6139193 7.5363145 7.48627   7.9148483
 7.973084  7.441485  7.3213663 7.73774   7.7424006 7.5285454 7.765645
 9.338779  7.815534  7.4064517 7.1331453 

5064 0.3341176425709444 0.5546695368364454
5435 0.3341176425709444 0.2235275403596461
5773 0.3341176425709444 0.19226204175502062
5948 0.3341176425709444 0.1508149268105626
[22.651194 22.60223  23.109688 22.19018   0.       23.748331 22.55284
 22.353245 22.30462   0.        0.        0.       22.752237 22.618483
 21.628513  0.       23.150677  0.        0.       22.317146 22.140577
 22.159685 22.758505  0.       22.56501  21.66246  23.025492 22.369436
  0.        0.       22.16373  22.220242  0.       22.95561  22.676126
 23.779028 23.299458 23.380522 23.564682 23.196993  0.       23.691746
  0.        0.        0.       22.44565  23.808422  0.       23.513159
 23.53759  23.168491  0.       22.870216 23.63182  24.171925 24.630163
 22.922531 23.038624 22.85493  23.590696  0.        0.       23.125708
 23.93695  23.758224  0.       23.038244  0.       23.513592 23.990059
 24.016678  0.       24.2207   23.608543  0.        0.       22.90482
  0.       22.938108  0.       23.80049  22.2601

9017 0.40470590030445774 0.004802769771777094
9131 0.40470590030445774 0.003859150648349896
9170 0.40470590030445774 0.005803618114441633
9219 0.40470590030445774 0.004171176656382158
9243 0.40470590030445774 0.004579934879438951
9295 0.40470590030445774 0.0038595223566517236
9341 0.40470590030445774 0.004600645444588736
9427 0.40470590030445774 0.004088901856448501
9463 0.40470590030445774 0.0034843518515117466
9484 0.40470590030445774 0.006802654330385849
9498 0.40470590030445774 0.005070173396961764
9532 0.40470590030445774 0.008034693432273343
9583 0.40470590030445774 0.006175665068440139
9597 0.40470590030445774 0.0046625531045719985
9611 0.40470590030445774 0.005427029164275154
9722 0.40470590030445774 0.0057470724859740585
9825 0.40470590030445774 0.004066300537670031
9900 0.40470590030445774 0.005726082448381931
9964 0.40470590030445774 0.005119836790254339
10021 0.40470590030445774 0.004594863398233428
10157 0.40470590030445774 0.0049659709446132185
[5.9960427 6.044503  5.8173

13420 0.5200000089757583 0.0008217704111302737
13490 0.5200000089757583 0.0013538757884816732
13582 0.5200000089757583 0.0011840992825455031
13765 0.5200000089757583 0.0015252423749188893
13821 0.5200000089757583 0.0022353803491569125
13922 0.5200000089757583 0.006141363619826734
14036 0.5200000089757583 0.0015694798188633286
14126 0.5200000089757583 0.0024584826634963973
14224 0.5200000089757583 0.0011480091015982907
14329 0.5200000089757583 0.002938992819690611
14423 0.5200000089757583 0.001496041075733956
14537 0.5200000089757583 0.0020832664173212833
14648 0.5200000089757583 0.0010008174082031473
14737 0.5200000089757583 0.004438212883542292
14825 0.5200000089757583 0.0038805809424957262
14928 0.5200000089757583 0.004180163063574582
15018 0.5200000089757583 0.0010153334980714135
15081 0.5200000089757583 0.001784048555418849
15180 0.5200000089757583 0.0016907058670767583
15246 0.5200000089757583 0.0017249244643608107
15336 0.5200000089757583 0.0022118883964139966
15389 0.52000000897

('CGCUCAGCCGCCCGGGCCGUUCAGGCCCGGGCGGCUGAGC', 0.5200000089757583)

In [73]:
batch_size = 1000
generations = 20
agent = RL_agent_DQN(wt, alphabet=RAA, gamma=0.8, batch_size=256, memory_size=10000, device=device)
agent.run_RL(generations=generations, train_epochs=400)

Q_Network(
  (linear1): Linear(in_features=160, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=40, bias=True)
  (bn3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=40, out_features=160, bias=True)
)
[0.07980745 0.09981431 0.10099482 0.0806646  0.         0.
 0.         0.09051907 0.         0.08890685 0.         0.
 0.         0.14231347 0.1229898  0.         0.17537147 0.
 0.         0.         0.         0.01885661 0.08652331 0.05594402
 0.10368662 0.         0.11089988 0.         0.         0.10319419
 0.         0.03334495 0.0620199  0.         0.1127741  0.07481047
 0.         0.         0.11987214 0.0159902  0.11037704 0.11625044
 0.         0.

[0.08081502 0.10445267 0.09950581 0.08644911 0.         0.
 0.         0.09028402 0.         0.0918102  0.         0.
 0.         0.14064285 0.12043548 0.         0.17264494 0.
 0.         0.         0.         0.01400212 0.0832102  0.05273647
 0.10164048 0.         0.10785906 0.         0.         0.11039177
 0.         0.03148001 0.06107474 0.         0.11140945 0.07736741
 0.         0.         0.1188916  0.01703386 0.1150726  0.11907925
 0.         0.         0.         0.13665615 0.         0.08860383
 0.         0.         0.         0.01057619 0.00899464 0.05845692
 0.         0.12095112 0.11646348 0.         0.14926697 0.
 0.         0.06420504 0.         0.11009144 0.0055927  0.08177567
 0.         0.         0.039531   0.02135615 0.         0.01424265
 0.09009241 0.10492362 0.12277291 0.         0.         0.
 0.         0.         0.         0.         0.1511192  0.12957413
 0.04970996 0.         0.06993157 0.         0.         0.0922221
 0.         0.         0.00070226 0.

[2.683873   2.8811362  2.8309076  3.0982625  0.         0.
 2.6900434  2.5673997  3.0546184  2.8972347  2.7098331  0.
 0.         2.7892132  2.788968   2.5882807  2.5801077  2.8373652
 2.607143   0.         0.07246052 2.8133783  2.873973   2.4818978
 2.640065   0.         2.5963142  0.         2.7742078  2.6866825
 0.         2.8779268  2.650395   0.         2.7272081  2.633561
 0.         2.463483   2.9744337  2.8653939  2.708384   2.7639103
 0.         3.2133822  2.5692751  2.9859173  3.153601   3.0821183
 0.         2.9378843  2.9889374  2.670598   3.0801718  2.7333188
 2.8000407  3.0217714  2.975326   0.         2.9781308  0.
 2.6443136  2.8118343  0.         2.9215827  2.89905    3.0186098
 2.6686084  2.5356865  2.596462   2.660975   2.7586482  2.655547
 2.8335476  2.904424   2.5859582  3.042049   0.         0.
 0.         2.7572389  0.         0.         2.6742096  2.6350706
 0.         2.75098    2.977734   2.6950896  3.3264673  2.6068547
 0.         0.         0.         2.8834

4314 0.33294116749482994 0.0034939520226907918
4772 0.33294116749482994 0.0038353306261706167
5186 0.33294116749482994 0.002689830114977667
[1.8900533 1.8865082 1.9622949 1.8366451 0.        1.872128  1.800961
 1.7948357 1.7222672 1.7383925 1.8608241 1.8846248 0.        1.9165517
 1.9236329 1.8097645 1.8728186 1.9170259 1.939867  0.        1.8843578
 1.9221321 1.9596493 1.8658419 1.8455324 0.        1.7984213 0.
 1.8719163 1.7472646 0.        1.7073528 1.8233483 0.        1.8330454
 1.8248734 1.918874  1.9837192 1.950817  1.9181972 1.9293914 2.0206528
 0.        2.028808  1.9458239 1.8675513 2.0223315 2.0010233 0.
 1.994867  2.037725  2.0713482 2.025038  2.00698   1.9487382 2.008465
 1.9420104 0.        1.9910257 0.        1.862677  1.928341  0.
 1.9881696 1.953767  2.091821  1.8804208 1.86597   1.8403893 1.7552071
 2.0014248 1.7893906 2.0036592 1.9174602 1.907692  2.0016632 1.9527502
 0.        0.        1.8991518 0.        1.9316537 1.992228  1.9143832
 1.7339176 2.099385  1.8704025 

6813 0.45294117647058824 0.002338349174387986
6959 0.45294117647058824 0.0026365510000323413
7119 0.45294117647058824 0.0030945940072706433
7267 0.45294117647058824 0.002614506736281328
7459 0.45294117647058824 0.003679804723651614
7589 0.45294117647058824 0.003101503825309919
7755 0.45294117647058824 0.0031001547812775244
7865 0.45294117647058824 0.0023224258034315426
7982 0.45294117647058824 0.001761819900275441
8099 0.45294117647058824 0.0018631226001889446
8225 0.45294117647058824 0.001396338316699257
8309 0.45294117647058824 0.0012844089551072102
[4.927724  4.812482  5.0628357 4.5385604 0.        4.3048463 4.585272
 4.2637477 4.538054  4.656479  4.748113  4.6997256 0.        4.770037
 4.383948  4.892731  4.6791368 4.7249427 4.856268  0.        4.836353
 4.8586392 4.745094  4.6144934 4.4621058 0.        4.487132  0.
 4.4449434 4.738899  0.        4.6957245 4.5718627 0.        4.459154
 4.4864626 4.5800943 4.6220393 4.7053733 4.976317  4.792811  4.822833
 0.        4.9643946 4.34237

9797 0.47529413559857536 0.0009999120997235878
9902 0.47529413559857536 0.0008859852230671094
10015 0.47529413559857536 0.0008823659950576257
10101 0.47529413559857536 0.0010597005939052906
10195 0.47529413559857536 0.0009612614873185521
[3.7600143 3.7531357 3.9276721 3.5753193 0.        3.5991483 3.5710232
 3.40942   3.605082  3.5895102 3.8614228 3.6264849 0.        3.4304686
 3.4905808 3.6581063 3.592267  3.7020648 3.7282417 0.        3.8486052
 3.822024  3.7193153 3.585792  3.5587156 0.        3.5485065 0.
 3.574664  3.6092753 0.        3.6578875 3.5554225 0.        3.5912378
 3.462015  3.6000547 3.049817  3.6337054 3.8365088 3.839281  3.8806722
 0.        3.7851965 3.4531567 3.6005785 4.074585  4.2117224 0.
 3.9922512 3.8873587 3.9831467 4.035372  4.193122  4.0095353 3.5584111
 3.5243025 0.        3.8917274 3.7956727 3.7851353 3.8088958 0.
 4.12534   4.001537  4.0924087 3.5008013 3.5800958 3.503055  3.5073671
 3.9698608 3.5722203 4.0429096 3.7115352 3.516962  4.1109095 3.9823935
 0

15009 0.5164706061868106 0.006111249168752692
15195 0.5164706061868106 0.0028895331698004156
15388 0.5164706061868106 0.002128842879028525
15533 0.5164706061868106 0.0022756720101460813
15669 0.5164706061868106 0.003270189075556118
15786 0.5164706061868106 0.001940260455448879
[3.8610897 3.856782  4.0125613 3.6811156 0.        3.7412405 3.7355165
 3.5909736 3.7208922 3.6362934 4.1196012 3.738386  3.802594  3.5747247
 3.6832035 3.6473038 3.7181227 3.7814267 3.8491232 3.9141467 3.893212
 3.8839188 3.8333337 3.6922112 3.659082  3.687254  3.6761305 3.7487426
 3.7278743 3.7162492 0.        3.7173576 3.6649203 3.8180637 3.7261326
 3.6887536 3.7349858 3.897206  3.8173063 3.9422646 3.8968737 3.9667792
 3.6668236 4.0197663 3.5879016 3.7054763 4.2246637 4.2589927 0.
 4.223505  3.8278465 4.090667  4.168732  4.2544546 4.1533213 3.572881
 3.8029823 0.        3.9813018 3.927694  3.8742216 3.9269052 0.
 4.1081734 4.187698  4.171232  3.6966217 3.665057  3.631502  3.6666389
 4.0954156 3.742352  4.16905

('GGUGCCGGCGCGGGGCCCGCAUAGGGCCCCGCGCCGGCAU', 0.5270588145536534)

In [38]:
batch_size = 1000
agent = RL_agent_DQN(wt, alphabet=RAA, gamma=0.8, memory_size=10000, device=device)
agent.run_RL(generations=20, train_epochs=40, epsilon_min=0.2)

Q_Network(
  (linear1): Linear(in_features=160, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=40, bias=True)
  (bn3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=40, out_features=160, bias=True)
)
[0.         0.         0.00943188 0.09440845 0.08236657 0.16149537
 0.04022703 0.0082418  0.07870349 0.         0.00418112 0.
 0.         0.         0.07941253 0.09027536 0.         0.
 0.         0.11003847 0.0043978  0.         0.         0.00227638
 0.         0.07191759 0.         0.12890662 0.09110717 0.11482675
 0.         0.         0.         0.10984014 0.05239729 0.0599668
 0.         0.04789581 0.         0.         0.12924725 0.10711745
 0.    

[0.         0.         0.01014275 0.09914748 0.08465273 0.16630806
 0.03929112 0.01011102 0.07966148 0.         0.00651534 0.
 0.         0.         0.0784291  0.08580427 0.         0.
 0.         0.11299177 0.00490537 0.         0.         0.
 0.         0.07123064 0.         0.12625246 0.08798044 0.11368578
 0.         0.         0.         0.111288   0.04895287 0.05729595
 0.         0.04862705 0.         0.         0.13263565 0.10582536
 0.         0.         0.         0.09538203 0.         0.14901178
 0.07161276 0.01624407 0.         0.06963313 0.         0.
 0.         0.         0.06677839 0.         0.         0.
 0.         0.         0.         0.0498019  0.03927595 0.06157403
 0.         0.04960936 0.         0.         0.01348276 0.
 0.         0.         0.16005982 0.         0.         0.02161822
 0.         0.13823788 0.         0.         0.         0.1378013
 0.08690245 0.         0.         0.         0.         0.02800744
 0.         0.06178055 0.         0.        

2714 0.2 0.002325622187345289
3327 0.2 0.0030109777697362007
[1.3797383  0.         1.3325628  1.4093516  1.3393365  1.3749008
 1.346369   1.3587296  1.3018328  1.2248678  1.3129864  1.2800211
 1.3702117  1.3087809  1.3167118  1.3698066  1.2841628  0.
 1.2518404  1.3411014  1.3114848  1.3235584  1.3050234  1.3263032
 1.2898694  1.3389105  0.         1.309613   1.3979731  1.2845199
 1.2875661  1.1958205  1.3855994  1.2739776  1.3648921  1.3821979
 1.3006551  1.315367   0.8701179  1.3211226  1.3035059  1.4839635
 1.255937   1.2909882  0.         1.3933322  0.92718786 1.4533875
 1.4683086  1.3726429  0.         1.3698537  1.4637406  0.
 1.2944524  1.4729773  1.3553636  0.         0.         0.
 1.3496884  0.         0.         1.4492762  1.415102   1.3758055
 0.         1.3858864  0.         0.         1.4449472  1.289467
 0.         1.4206184  1.3412685  1.3391305  1.3210707  1.3999791
 0.         1.2856541  0.         1.0091491  0.         1.4910445
 1.3645433  0.         1.2673465  1.2

[1.3011351 0.        1.3592889 1.3618174 1.3039597 1.2984653 1.3393934
 1.329717  1.25339   1.2897276 1.3652086 1.2167455 1.3216522 1.4841082
 1.2530614 1.3005884 1.3972726 0.        1.3209059 1.3521909 1.3069865
 1.3643427 1.3312618 1.306137  1.3030894 1.2583745 0.        1.2148527
 1.3085116 1.1497308 1.3648131 1.2646118 1.4000232 1.3580894 1.2573017
 1.3366362 1.2693777 1.2529097 1.3490518 1.3568591 1.3896992 1.3962679
 1.345718  1.3560188 0.        1.4198971 1.3925397 1.3503494 1.3817605
 1.3704462 0.        1.4286343 1.4933146 0.        1.4028308 1.4805058
 1.319242  0.        0.        0.        1.3137121 0.        0.
 1.3831896 1.3875891 1.3094605 0.        1.4375309 0.        0.
 1.3971314 1.3671409 0.        1.3781816 1.3316221 1.3403537 1.4097372
 1.3673667 0.        1.3669301 0.        1.3562528 0.        1.4404765
 1.378981  0.        1.3360093 1.2752857 0.        1.2359388 0.
 1.2837931 1.2832942 0.        1.3524002 0.        1.3920348 1.405981
 1.43567   1.3920059 1.38823

6323 0.35647057925953585 0.0015602064711856655
6620 0.35647057925953585 0.0017163074167910962
6925 0.35647057925953585 0.003666347108082846
7254 0.35647057925953585 0.00283857139875181
[2.0834653 0.        2.085692  2.0042458 2.080572  2.0107687 1.9145225
 2.1030593 2.0305483 2.0033002 2.1424224 2.0778604 2.045483  2.1346807
 1.9988911 2.0970805 2.0011978 1.304854  2.0949016 1.990887  2.006841
 2.1875582 2.1022074 2.0411606 2.0219166 2.004401  0.        2.032289
 2.0361712 1.9861785 2.0828707 2.1046612 2.2518466 2.0200863 2.0057056
 2.067064  1.994215  2.0262222 1.9709067 2.114152  2.1537766 2.2705028
 2.1385536 2.0974927 0.        2.2661934 2.2346833 2.1676118 2.2069852
 2.2528954 0.        2.2238717 2.2227383 0.        2.2137933 2.2525654
 2.032025  0.        0.        0.        2.0532203 0.        0.
 2.1525838 2.2195063 2.3503995 0.        2.2459993 0.        0.
 2.1833465 2.122421  0.        2.0838854 2.0521894 2.1204994 2.286398
 2.1269028 0.        2.1499155 2.0287468 2.0285525 

7799 0.4 0.004016830379259773
8103 0.4 0.003602199419401586
8362 0.4 0.003208036470459774
[2.6371915 0.        2.6078072 2.5540082 2.6504545 2.487643  2.4223611
 2.5126786 2.515955  2.5131905 2.6862123 2.51708   2.5965698 2.6618016
 2.582755  2.5129147 2.508926  2.5489988 2.6349282 2.4548967 2.549761
 2.79071   2.6876488 2.46275   2.486627  2.4435246 0.        2.5405953
 2.6051464 2.5818005 2.6959424 2.6077902 2.8871996 2.4800212 2.453786
 2.617607  2.594391  2.525137  2.4830766 2.6558297 2.7113254 2.7302413
 2.7492957 2.7746854 0.        2.778161  2.8102791 2.7726557 2.7566912
 2.7461202 0.        2.7857325 2.7289374 0.        2.8757317 2.8648925
 2.4680302 0.        0.        0.        2.5663714 0.        0.
 2.7887695 2.81991   2.9238453 0.        2.8657205 0.        2.532151
 2.7071493 2.60997   0.        2.5762827 2.4373786 2.6356592 2.8344789
 2.610786  0.        2.678467  2.6094577 2.5382555 2.5932808 2.630432
 2.791454  0.        2.489363  2.5886793 2.3759189 2.6020114 0.
 2.71

10026 0.4317647148581112 0.0054402679263148455
10272 0.4317647148581112 0.00394937505479902
10471 0.4317647148581112 0.004107160150306299
10730 0.4317647148581112 0.004842862294754014
11040 0.4317647148581112 0.005309851793572307
11293 0.4317647148581112 0.0044801300740800794
11578 0.4317647148581112 0.008144412655383348
11784 0.4317647148581112 0.007411241962108761
12031 0.4317647148581112 0.0023385304113617165
12275 0.4317647148581112 0.0031343657668912784
12559 0.4317647148581112 0.005773932868032716
12708 0.4317647148581112 0.0035289877880131824
12935 0.4317647148581112 0.005416404447169043
13158 0.4317647148581112 0.003270380839239806
13323 0.4317647148581112 0.0039093873434467245
13551 0.4317647148581112 0.007229463034309447
13874 0.4317647148581112 0.00924281794577837
14061 0.4317647148581112 0.009659996564732865
14282 0.4317647148581112 0.008353234879905358
14545 0.4317647148581112 0.006965321043389849
[3.5316925 0.        3.4703312 3.334074  3.603795  3.4238205 3.3048751
 3.34

('GGGGCGGGAGAGGACGCCCAGUAGGGGGCCUCUCCCGCCC', 0.4388235204360064)

In [39]:
batch_size = 1000
generations = 20
agent = RL_agent_DQN(wt, alphabet=RAA, gamma=0.9, memory_size=10000, device=device)
agent.run_RL(generations=20, train_epochs=40)

Q_Network(
  (linear1): Linear(in_features=160, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=40, bias=True)
  (bn3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=40, out_features=160, bias=True)
)
[0.         0.         0.04841536 0.         0.06368769 0.03880617
 0.1708372  0.06790138 0.         0.         0.10455563 0.06122993
 0.02910596 0.         0.         0.         0.06646234 0.03728576
 0.14601626 0.00498905 0.07405932 0.05907581 0.         0.05444898
 0.1233772  0.         0.11450447 0.15579537 0.1259845  0.0333193
 0.         0.         0.         0.         0.         0.
 0.         0.04292832 0.         0.09007409 0.12635419 0.0832317


[0.         0.         0.04911017 0.         0.06300279 0.03701607
 0.17018966 0.0657393  0.         0.         0.10589771 0.06210753
 0.02556268 0.         0.         0.         0.06815919 0.03578322
 0.14928988 0.00334537 0.07206579 0.06110452 0.         0.06085588
 0.12669876 0.         0.11469334 0.15219215 0.1239157  0.03242629
 0.         0.         0.         0.         0.         0.
 0.         0.04045434 0.         0.09166113 0.12412737 0.08276865
 0.         0.         0.         0.11120061 0.12330455 0.
 0.         0.03347604 0.06318808 0.12050883 0.         0.14565851
 0.07737547 0.         0.         0.15928671 0.         0.04087942
 0.         0.17887692 0.04203531 0.         0.         0.03194004
 0.         0.11326367 0.03035538 0.05134697 0.         0.
 0.         0.04963373 0.09352957 0.14170563 0.         0.0738268
 0.         0.02658218 0.07887706 0.         0.03176539 0.
 0.         0.         0.10204703 0.03554232 0.         0.
 0.04902081 0.         0.07419488 0.

3445 0.2694117602180032 0.015594095760025083
3996 0.2694117602180032 0.018576975038740784
[0.         2.5758145  3.110498   3.3743594  3.18722    3.189009
 2.9980278  3.292468   0.0225013  0.         3.001958   3.1868725
 3.2863035  3.3087788  0.         0.         3.2488587  3.2585013
 3.0172825  3.2900882  3.2720487  3.0657458  3.2027755  3.1990936
 3.081877   0.         3.0906742  3.0870497  3.2399266  3.1825697
 3.2500722  0.         0.         3.2320018  0.         0.
 3.317058   3.2106373  3.2410579  3.2516503  3.2075903  3.5000632
 0.         0.         3.3990884  3.2137153  3.4023173  3.3728304
 3.3652673  0.         3.2012835  3.4130018  0.         3.1769109
 3.3979678  0.         3.3781776  3.340909   3.252319   3.3187594
 0.         3.35543    1.9445009  0.         3.1414165  3.2440984
 3.454279   3.309533   3.3627427  3.3423781  3.3494582  3.51289
 3.2874868  3.3155742  2.9692779  3.4368742  3.2627523  3.153468
 0.         3.071709   3.3000555  0.         3.2884393  0.
 3.2

4962 0.36470588235294116 0.012563292495906353
5317 0.36470588235294116 0.016511564166285096
5649 0.36470588235294116 0.009554687334457412
5993 0.36470588235294116 0.017325855570379644
6292 0.36470588235294116 0.018219729932025076
6516 0.36470588235294116 0.02432465013116598
[0.        7.52079   7.511512  7.9263763 7.4146075 7.4599032 7.4350142
 7.68739   7.5375633 0.        7.794269  7.6292887 7.6916065 7.7302804
 7.686066  0.        7.634936  7.7239795 7.367268  7.5024905 7.832654
 7.649899  7.670606  8.195563  7.4697723 0.        7.3690586 7.719405
 7.516364  7.368003  7.4614277 0.        0.        7.665069  0.
 0.        7.586898  7.666483  7.594132  7.784319  8.057067  7.925689
 0.        0.        7.7616262 7.98006   7.8785434 7.9867883 7.7900305
 0.        7.6116114 8.016183  5.487672  7.677958  7.6440125 0.
 7.979759  7.786057  7.537245  7.633213  0.        8.003398  7.3928795
 0.        7.584574  7.8723755 7.9756947 7.8539486 7.9250684 7.8579316
 7.6551466 8.044124  7.7909236 7

6970 0.45411762910730696 0.020733187510631978
7251 0.45411762910730696 0.028128231561277063
7502 0.45411762910730696 0.019017234491184355
7657 0.45411762910730696 0.01737105577485636
7795 0.45411762910730696 0.04629292953759432
7996 0.45411762910730696 0.056120811228174715
8160 0.45411762910730696 0.054217913164757194
8282 0.45411762910730696 0.05344603057019413
8472 0.45411762910730696 0.09848140110261738
8648 0.45411762910730696 0.05992683432996273
8781 0.45411762910730696 0.04137012241408229
8883 0.45411762910730696 0.25689512249082325
9060 0.45411762910730696 0.06421370664611459
9243 0.45411762910730696 0.21979930456727742
9399 0.45411762910730696 0.06760943406261503
9567 0.45411762910730696 0.07807816374115646
9721 0.45411762910730696 0.09293549535796046
9903 0.45411762910730696 0.1070060391444713
10080 0.45411762910730696 0.15307616954669356
10254 0.45411762910730696 0.10891328658908606
10445 0.45411762910730696 0.130052306689322
10609 0.45411762910730696 0.15429032370448112
1078

('GGAUAGGGGCCGCCCGCCCCCGCUGGGGGGGGGGCGGCCC', 0.48823529411764705)

In [75]:
# control, simple greedy 
batch_size = 1000
agent = RL_agent_DQN(wt, alphabet=RAA, gamma=0, memory_size=10000, device=device)
agent.run_RL(generations=20, train_epochs=40, epsilon_min=0.2)

Q_Network(
  (linear1): Linear(in_features=160, out_features=160, bias=True)
  (bn1): BatchNorm1d(160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear2): Linear(in_features=160, out_features=40, bias=True)
  (bn2): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear3): Linear(in_features=40, out_features=40, bias=True)
  (bn3): BatchNorm1d(40, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear4): Linear(in_features=40, out_features=160, bias=True)
)
[0.1766599  0.         0.1455445  0.         0.         0.00862863
 0.         0.         0.         0.09159827 0.         0.
 0.16137333 0.12944189 0.04122588 0.         0.         0.08419764
 0.         0.0339617  0.         0.01635132 0.12183675 0.
 0.         0.08280537 0.06491239 0.         0.         0.0021084
 0.         0.07488443 0.         0.06782793 0.         0.
 0.0098681  0.         0.         0.04546724 0.         0.00559301
 0.13941436 0. 

[0.17773326 0.         0.14683044 0.         0.         0.00723547
 0.         0.         0.         0.09278361 0.         0.
 0.16079612 0.13010336 0.03547945 0.         0.         0.08255832
 0.         0.03626961 0.         0.0166194  0.12062386 0.
 0.         0.08428322 0.06482885 0.         0.         0.00139686
 0.         0.07792129 0.         0.07122925 0.         0.
 0.01018019 0.         0.         0.0446902  0.         0.005616
 0.14170618 0.         0.         0.13889319 0.17410439 0.
 0.         0.06821947 0.         0.04903721 0.         0.
 0.10388152 0.1263671  0.17260551 0.         0.         0.
 0.         0.         0.         0.13890336 0.11236906 0.
 0.         0.         0.         0.         0.         0.07211619
 0.16854726 0.12236734 0.         0.         0.11241233 0.
 0.         0.         0.         0.         0.03206398 0.16938749
 0.13337864 0.         0.         0.11164962 0.05050375 0.04957928
 0.         0.09012394 0.         0.         0.08037797 0.086

728 0.1411764705882353 0.0021706992643885314
[0.06530215 0.         0.05593719 0.         0.         0.03820788
 0.         0.00597138 0.         0.04550094 0.         0.
 0.06516643 0.09682768 0.08331591 0.         0.         0.04580618
 0.         0.07385804 0.         0.0513395  0.08178003 0.
 0.         0.07524087 0.0789147  0.         0.         0.06102842
 0.01412127 0.05859244 0.         0.06372697 0.         0.
 0.03062132 0.         0.         0.02938696 0.         0.01356933
 0.07538229 0.         0.         0.0633311  0.09097052 0.
 0.         0.05114135 0.         0.06442192 0.         0.
 0.08793539 0.06475474 0.09623241 0.         0.01433004 0.
 0.         0.         0.         0.04723193 0.07294907 0.
 0.0281753  0.01661227 0.         0.         0.         0.0715635
 0.07062184 0.06261116 0.         0.         0.06311883 0.
 0.         0.         0.         0.         0.08247781 0.07546389
 0.09507415 0.         0.         0.04946991 0.04841335 0.04438864
 0.         0.0

2814 0.22470588684082032 0.0008001939801033586
[0.14158256 0.         0.11169489 0.         0.         0.13548037
 0.         0.08602229 0.         0.12117844 0.         0.
 0.1424621  0.12594365 0.11606689 0.         0.         0.14019585
 0.01267108 0.12884957 0.15490702 0.07047278 0.0743943  0.
 0.         0.09815405 0.10995176 0.         0.         0.11945304
 0.         0.11199883 0.         0.11267278 0.         0.
 0.09941149 0.         0.         0.11523282 0.         0.05041057
 0.14775425 0.         0.         0.12645738 0.09952056 0.
 0.         0.10512932 0.0763176  0.13028231 0.         0.
 0.11390214 0.07789053 0.07723476 0.         0.06421509 0.
 0.         0.         0.         0.10924688 0.15167901 0.
 0.09593747 0.0620524  0.         0.         0.         0.15242119
 0.15222901 0.11552221 0.         0.         0.1498122  0.
 0.         0.         0.         0.         0.10667259 0.15989092
 0.09745563 0.         0.         0.10497172 0.12223034 0.15208687
 0.         

3988 0.3247058868408203 0.0005944777883996721
4564 0.3247058868408203 0.0005180915955861565
5062 0.3247058868408203 0.0005117118271300569
5521 0.3247058868408203 0.0004894283105386421
[0.20983694 0.         0.21256721 0.         0.         0.21082509
 0.         0.22975674 0.         0.18149637 0.         0.
 0.20341817 0.19579473 0.1725741  0.         0.         0.20070922
 0.2152683  0.23909119 0.23344722 0.19273776 0.07378875 0.
 0.         0.19095044 0.21269068 0.19340375 0.         0.2220583
 0.         0.18619925 0.         0.18722302 0.         0.
 0.2255787  0.         0.         0.20494612 0.         0.0869246
 0.22149616 0.         0.         0.23042828 0.22321169 0.
 0.         0.22529924 0.22381714 0.19621392 0.         0.
 0.20932901 0.20154503 0.23668212 0.         0.19775188 0.
 0.00191334 0.         0.         0.23321426 0.256563   0.
 0.21183467 0.22168854 0.         0.20820585 0.         0.24032073
 0.22379373 0.24814095 0.         0.         0.23027426 0.
 0.        

[0.30303085 0.11238243 0.28804785 0.         0.         0.28134248
 0.         0.33379507 0.         0.24817082 0.         0.
 0.27146736 0.2692142  0.2997188  0.         0.         0.30655
 0.29470634 0.32222605 0.3143473  0.3242062  0.04698741 0.
 0.         0.26806754 0.29952133 0.2594121  0.         0.29379168
 0.         0.2217118  0.         0.23821712 0.         0.
 0.29609045 0.         0.         0.31223172 0.         0.2785195
 0.25470194 0.         0.         0.327021   0.34520954 0.
 0.         0.34346455 0.34077486 0.2696723  0.         0.0335734
 0.2909379  0.30791077 0.3264935  0.         0.30307215 0.
 0.33148187 0.         0.         0.3452113  0.35823143 0.
 0.28900966 0.33921313 0.         0.2885842  0.         0.3467818
 0.34435785 0.34238917 0.         0.         0.30309537 0.
 0.         0.29182777 0.30342138 0.         0.33480048 0.35133648
 0.33487716 0.24662994 0.         0.2946197  0.35739276 0.28547668
 0.         0.3241267  0.3367674  0.         0.27217913 0

10466 0.4141176560345818 0.0003067196248593973
10612 0.4141176560345818 0.00023242349889187607
10829 0.4141176560345818 0.0002241696674900595
10972 0.4141176560345818 0.0002748781509581022
11144 0.4141176560345818 0.0002647532517585205
11278 0.4141176560345818 0.00018910931394202634
11392 0.4141176560345818 0.00017582734180905392
11526 0.4141176560345818 0.0002648048233822919
11657 0.4141176560345818 0.00023745310099911875
11723 0.4141176560345818 0.00022669822246825788
11865 0.4141176560345818 0.0002231408661828027
12036 0.4141176560345818 0.00020708761585410684
12174 0.4141176560345818 0.0002103564131175517
12326 0.4141176560345818 0.00020408568198035937
12445 0.4141176560345818 0.00019110023677058053
12588 0.4141176560345818 0.00018050356393359835
12763 0.4141176560345818 0.00018447493093844968
12856 0.4141176560345818 0.0001728698725855793
12938 0.4141176560345818 0.00017309020040556788
13057 0.4141176560345818 0.00017172032676171511
13218 0.4141176560345818 0.00016790617883088998


('UGCCCGGUCGGCCAACGGGAGAUGGCCGACUGGGCAACAG', 0.4141176560345818)